c:\Users\GERARDITO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


c:\Users\GERARDITO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
Int64Index: 481672 entries, 0 to 481671
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date_sales      481672 non-null  object 
 1   account_name    481672 non-null  object 
 2   store_name      481672 non-null  object 
 3   sku             481672 non-null  object 
 4   sell_out_units  481672 non-null  float64
 5   sell_out_value  481672 non-null  float64
 6   stock_units     481672 non-null  float64
 7   stock_value     481672 non-null  float64
 8   account_id      481672 non-null  int64  
dtypes: float64(4), int64(1), object(4)
memory usage: 36.7+ MB
None
